### Deteção de anomalias em dados de gestão de resíduos em Portugal

##### João Ribeiro

## <span style="color:Darkgreen"> Tratamento e Análises  </span> 

---

### <span style="color:Darkgreen"> Leitura do dataset  </span> 

In [ ]:
# Bibliotecas
import pandas as pd
import numpy as np
import warnings
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
from sklearn.metrics import silhouette_score
warnings.filterwarnings("ignore")

In [ ]:
df_inicial = pd.read_csv('view_egar_guias_filtrado.csv', sep=';')

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_inicial.head())

In [ ]:
df_inicial.shape

### <span style="color:Darkgreen"> Tratamento  </span> 

In [ ]:
column_names = df_inicial.columns.tolist()
print(column_names)

In [ ]:
num_linhas = df_inicial.shape[0]
print("Número de linhas:", num_linhas)

In [ ]:
df_inicial.isna().sum()

In [ ]:
df_inicial.dtypes

In [ ]:
# Variável Produtor Origem
categorias_produtor_origem = df_inicial['produtor_origem'].value_counts()
print("Categorias da variável produtor_origem:")
print(categorias_produtor_origem)

In [ ]:
# Variável qualidade
categorias_qualidade = df_inicial['qualidade'].value_counts()
print("Categorias da variável qualidade:")
print(categorias_qualidade)

In [ ]:
df = df_inicial.drop(columns=['fk_guia_acompanhamento_residuos','egar','validacao','criada_por', 'produtor_local','produtor_origem','qualidade','data_estado',
                      'dados_transportadores','destinatario_estabelecimento','utilizador_criacao','utilizador_alteracao','grupo_hospitalar',
                      'num_plano_gestao_lamas','num_plano_gestao_lamas_recebido'])

In [ ]:
column_names = df.columns.tolist()
print(column_names)

In [ ]:
df.isna().sum()

In [ ]:
df['ler_recebido_codigo'] = df['ler_recebido_codigo'].fillna(df['ler_codigo'])
df = df.dropna(subset=['quantidade'])
df = df.dropna(subset=['produtor_nif'])
df = df.dropna(subset=['tipo_op_codigo'])
df['tipo_op_recebido_codigo'] = df['tipo_op_recebido_codigo'].fillna(df['tipo_op_codigo'])
df['quantidade_recebida'] = df['quantidade_recebida'].fillna(df['quantidade'])

In [ ]:
# Verificar se há muitos codigos ler recebidos diferentes dos que estão em estudo
categorias_ler_recebido_codigo = df['ler_recebido_codigo'].value_counts()
print("Categorias da variável ler_recebido_codigo:")
print(categorias_ler_recebido_codigo)

In [ ]:
# Valores únicos de 'ler_codigo'
codigos_unicos = df['ler_codigo'].unique()  # Supondo que você quer os 3 primeiros valores únicos
codigos_unicos

In [ ]:
# Filtrar os registos'ler_recebido_codigo' que não são iguais a nenhum dos códigos'ler_codigo'
registos_diferentes = df[~df['ler_recebido_codigo'].isin(codigos_unicos)]
total_registos = len(df['ler_recebido_codigo'])
contagem_diferentes = len(registos_diferentes)
percentagem_diferentes = (contagem_diferentes / total_registos) * 100
print(f"Percentagem de registos diferentes entre ler_codigo e ler_codigo_recebido: {percentagem_diferentes:.2f}%")
print(f"Número de linhas que correspondem aos registos diferentes: {contagem_diferentes}")

In [ ]:
# Valores únicos de 'ler_codigo'
codigos_unicos = df['ler_recebido_codigo'].unique()
codigos_unicos

In [ ]:
codigos_filtrar = [150106, 150105, 150110]
df = df[df['ler_recebido_codigo'].isin(codigos_filtrar)]

In [ ]:
df = df.drop(columns=['ler_perigosidade','ler_recebido_perigosidade'])

In [ ]:
num_colunas = df.shape[1]
print(f'O número de colunas é: {num_colunas}')

In [ ]:
# Substituir os valores da coluna 'ler_codigo'
df['ler_codigo'] = df['ler_codigo'].replace({
    150105: 'Compósitos',
    150106: 'Misturas',
    150110: 'Embalagens c/ resíduos ou contaminadas'
})

In [ ]:
# Substituir os valores da coluna 'ler_recebido_codigo'
df['ler_recebido_codigo'] = df['ler_recebido_codigo'].replace({
    150105: 'Compósitos',
    150106: 'Misturas',
    150110: 'Embalagens c/ resíduos ou contaminadas'
})

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df.head())

In [ ]:
df.shape

## <span style="color:Darkgreen"> Análise de variáveis relevantes  </span> 

#### <span style="color:Darkgreen"> Variável Estado  </span> 

In [ ]:
# Variável estado
categorias_estado = df['estado'].value_counts()
print("Categorias da variável estado:")
print(categorias_estado)

In [ ]:
# Gerar a tabela de percentagens entre 'ler_codigo' e 'estado'
tabela_estado_percentagens = pd.crosstab(df['ler_codigo'], df['estado'], normalize='index') * 100
tabela_estado_percentagens = tabela_estado_percentagens.round(2)
print("Tabela % 'estado' e 'ler_codigo':")
tabela_estado_percentagens

#### <span style="color:Darkgreen">Variável mercadorias perigosas </span> 

In [ ]:
# Variável mercadorias_perigosas
categorias_mercadorias_perigosas = df['mercadorias_perigosas'].value_counts()
print("Categorias da variável mercadorias perigosas:")
print(categorias_mercadorias_perigosas)

In [ ]:
# Tabela % 'mercadorias_perigosas' e 'ler_codigo'
tabela_mercadperigosas_percentagens = (pd.crosstab( df['ler_codigo'],df['mercadorias_perigosas'],normalize='index') * 100).round(2)
print("Tabela %'mercadorias_perigosas' e 'ler_codigo':")
tabela_mercadperigosas_percentagens

#### <span style="color:Darkgreen">Variável Quantidade </span> 

In [ ]:
# Agrupar por codigo e somar quantidades de cada um
quantidade_total_por_codigo = df.groupby('ler_codigo')['quantidade'].sum()

# Quantidade total produzida para cada código
print("Quantidade total produzida para cada código:")
print(quantidade_total_por_codigo)

In [ ]:
# Converter de quilogramas para toneladas
quantidade_total_por_codigo_toneladas = (quantidade_total_por_codigo / 1000).round(2)
print("Quantidade total produzida para cada código em toneladas:")
print(quantidade_total_por_codigo_toneladas)

In [ ]:
# Agrupar por codigo e somar quantidades recebidas de cada um
quantidade_recebida_por_codigo = df.groupby('ler_recebido_codigo')['quantidade_recebida'].sum()

# Quantidade total produzida para cada código
print("Quantidade recebida para cada código:")
print(quantidade_recebida_por_codigo)

In [ ]:
# Converter de quilogramas para toneladas quantidade recebida
quantidade_recebida_por_codigo_toneladas = (quantidade_recebida_por_codigo / 1000).round(2)
print("Quantidade recebida para cada código em toneladas:")
print(quantidade_recebida_por_codigo_toneladas)

In [ ]:
df_filt = df[df['estado'] == 'Concluída (certificado de receção)']
df_filt['quantidade_toneladas'] = df_filt['quantidade'] / 1000
df_filt['quantidade_recebida_toneladas'] = df_filt['quantidade_recebida'] / 1000
df_filt['diferenca'] = df_filt['quantidade_toneladas'] != df_filt['quantidade_recebida_toneladas']
df_diff = df_filt.groupby('ler_codigo').agg(
    diferenca_toneladas=('quantidade_toneladas', lambda x: x.sum() - df_filt.loc[x.index, 'quantidade_recebida_toneladas'].sum()),
    operacoes_diferenca=('diferenca', 'sum')  # Conta o número de operações com diferenças
).reset_index()

df_diff

In [ ]:
plt.figure(figsize=(10, 8))
sns.boxplot(x='ler_codigo', y='diferenca_toneladas', data=df_filt, color='#026460')
plt.ylim(-30, 55)  
plt.xlabel('Tipo de Resíduo', fontsize=12)
plt.ylabel('Diferença em Toneladas', fontsize=12)
plt.xticks(rotation=0)
plt.tight_layout()
plt.grid(True, axis='y', linestyle='--', linewidth=0.5)
plt.show()

In [ ]:
df_filt2 = df[df['estado'] == 'Corrigida']
df_filt2['quantidade_toneladas'] = df_filt2['quantidade'] / 1000
df_filt2['quantidade_recebida_toneladas'] = df_filt2['quantidade_recebida'] / 1000
df_filt2['diferenca'] = df_filt2['quantidade_toneladas'] != df_filt2['quantidade_recebida_toneladas']
df_diff2 = df_filt2.groupby('ler_codigo').agg(
    diferenca_toneladas=('quantidade_toneladas', lambda x: x.sum() - df_filt2.loc[x.index, 'quantidade_recebida_toneladas'].sum()),
    operacoes_diferenca=('diferenca', 'sum') 
).reset_index()
df_diff2

In [ ]:
plt.figure(figsize=(10, 8))
sns.boxplot(x='ler_codigo', y='diferenca_toneladas', data=df_filt2, color='#026460')
plt.ylim(-15, 10)
plt.xlabel('Tipo de Resíduo', fontsize=12)
plt.ylabel('Diferença em Toneladas', fontsize=12)
plt.xticks(rotation=0)
plt.tight_layout()
plt.grid(True, axis='y', linestyle='--', linewidth=0.5)
plt.show()

In [ ]:
df_filt = df[df['estado'] == 'Concluída (certificado de receção)']
df_count = df_filt.groupby('ler_codigo').size().reset_index(name='numero_de_registos')
df_count

#### <span style="color:Darkgreen">Variável Nome do Produtor  </span> 

In [ ]:
# Top 10 dos produtores que aparecem mais vezes entre 2019 e 2022
produtor_top10 = df['produtor_nome'].value_counts().head(10)
print("Top 10 produtores:")
produtor_top10

In [ ]:
# # Top 5 de produtores por codigo
pd.set_option('display.max_colwidth', None)
contagem_produtores = (df.groupby(['ler_codigo', 'produtor_nome']).size().reset_index(name='count'))
contagem_produtores = contagem_produtores.sort_values(['ler_codigo', 'count'], ascending=[True, False])
top_produtores = contagem_produtores.groupby('ler_codigo').head(5)
print("Top 5 de produtores que aparecem por código:")
top_produtores

#### <span style="color:Darkgreen"> Variável Nome do Destinatário</span> 

In [ ]:
# Top 10 dos tranformadores que aparecem mais vezes entre 2019 e 2022
transformador_top10 = df['destinatario_nome'].value_counts().head(10)
print("Top 10 transformadores:")
print(transformador_top10)

In [ ]:
# Top 5 de transformadores por codigo
pd.set_option('display.max_colwidth', None)
contagem_transformadores = (df.groupby(['ler_codigo', 'destinatario_nome']).size().reset_index(name='count'))
contagem_transformadores = contagem_transformadores.sort_values(['ler_codigo', 'count'], ascending=[True, False])
top_transformadores = contagem_transformadores.groupby('ler_codigo').head(5)
print("Top 5 de transformadores que aparecem por código:")
top_transformadores

In [ ]:
# Produtores e destinatários
df_prod_dest = df[df['estado'] == 'Concluída (certificado de receção)']
produtores = set(df_prod_dest['produtor_nome'].dropna().unique())
destinatarios = set(df_prod_dest['destinatario_nome'].dropna().unique())
produtores_e_destinatarios = produtores.intersection(destinatarios)
print("Produtores que também são destinatários:")
for nome in produtores_e_destinatarios:
    print(nome)

In [ ]:
# Contar quantos produtores são também destinatários
count_produtores_e_destinatarios = len(produtores_e_destinatarios)
print("\nQuantidade de produtores que também são destinatários:",count_produtores_e_destinatarios)

#### <span style="color:Darkgreen"> Média do início e fim de transporte</span> 

In [ ]:
# Converter colunas para datetime
df['data_inicio_transporte'] = pd.to_datetime(df['data_inicio_transporte'])
df['data_fim_transporte'] = pd.to_datetime(df['data_fim_transporte'])

# Filtrar registos que têm data_fim_transporte não nulo
df_filtrado = df.dropna(subset=['data_fim_transporte'])

# Diferença entre data_fim_transporte e data_inicio_transporte
df_filtrado['diferenca_tempo'] = df_filtrado['data_fim_transporte'] - df_filtrado['data_inicio_transporte']

# Extrair dias, horas e minutos da diferença de tempo
df_filtrado['horas'] = df_filtrado['diferenca_tempo'].dt.components['hours']
df_filtrado['minutos'] = df_filtrado['diferenca_tempo'].dt.components['minutes']

# Calcular o total de horas e minutos
df_filtrado['total_minutos'] = df_filtrado['horas'] * 60 + df_filtrado['minutos']

df_media_tempo = df_filtrado.groupby('ler_codigo')['total_minutos'].mean().reset_index()
df_media_tempo['horas'] = df_media_tempo['total_minutos'] // 60
df_media_tempo['minutos'] = df_media_tempo['total_minutos'] % 60
df_media_tempo['media_tempo'] = df_media_tempo.apply(lambda row: f"{int(row['horas'])}h{int(row['minutos'])}min", axis=1)
df_media_tempo.drop(['total_minutos', 'horas', 'minutos'], axis=1, inplace=True)
print(df_media_tempo)


In [ ]:
# Boxplot whiskers - distribuição dos tempos de transporte por tipo de resíduo
plt.figure(figsize=(10, 6))
df_filtrado.boxplot(column='horas', by='ler_codigo', grid=False)
plt.xlabel('Tipo de Resíduo')
plt.ylabel('Total de Horas')
labels = ['Compósitos', 'Contaminadas', 'Misturas']
plt.xticks(ticks=[1, 2, 3], labels=labels, rotation=45, ha='right')
plt.tight_layout() 
plt.show()

#### <span style="color:Darkgreen"> Análise por Anos </span> 

In [ ]:
# Coluna 'data_criacao' para datetime
df['data_criacao'] = pd.to_datetime(df['data_criacao'])

# Extrair o ano da coluna 'data_criacao'
df['ano'] = df['data_criacao'].dt.year
df['quantidade_toneladas'] = df['quantidade'] / 1000
df['quantidade_recebida_toneladas'] = df['quantidade_recebida'] / 1000

# Agrupar por ano e código LER e soma das quantidades enviadas
agg_df = df.groupby(['ano', 'ler_codigo']).agg({
    'quantidade_toneladas': 'sum'
}).reset_index()

palette = sns.color_palette("dark:#5A9_r", n_colors=5)
plt.figure(figsize=(10, 6))

top_ler_codigos = agg_df['ler_codigo'].value_counts().index[:5]
for i, ler_codigo in enumerate(top_ler_codigos):
    subset = agg_df[agg_df['ler_codigo'] == ler_codigo]
    plt.plot(subset['ano'], subset['quantidade_toneladas'], marker='o', color=palette[i], label=f'{ler_codigo}')

plt.title('Evolução da Quantidade Enviada ao Longo dos Anos por Tipo de Resíduo', fontsize=14)
plt.xlabel('Ano', fontsize=12)
plt.ylabel('Quantidade (Toneladas)', fontsize=12)
plt.xticks([2019, 2020, 2021, 2022], fontsize=10)
plt.yticks(fontsize=10)
plt.grid(True, which='both', linestyle='--', linewidth=0.5, axis='y') 
plt.legend(title='Tipo de Resíduo', fontsize=10, title_fontsize=12, loc='center left', bbox_to_anchor=(1.1, 0.5))
sns.despine()
plt.show()
# Exibir os valores exatos para cada ler_codigo ao longo dos anos
for ler_codigo in agg_df['ler_codigo'].unique():
    print(f"\nValores para o código LER: {ler_codigo}")
    print(agg_df[agg_df['ler_codigo'] == ler_codigo][['ano', 'quantidade_toneladas']])

In [ ]:
# Calcular a diferença de quantidade em relação ao ano anterior e a % de crescimento
agg_df['pct_change'] = agg_df.groupby('ler_codigo')['quantidade_toneladas'].pct_change() * 100

# Tabela com as percentagens de crescimento/descida
tabela_pct_change = agg_df[['ano', 'ler_codigo', 'quantidade_toneladas', 'pct_change']].dropna()

tabela_pct_change

In [ ]:
# Agrupar por ano e código LER e soma das quantidades recebidas
agg_df = df.groupby(['ano', 'ler_codigo']).agg({
    'quantidade_recebida_toneladas': 'sum'
}).reset_index()

palette = sns.color_palette("dark:#5A9_r", n_colors=5)
plt.figure(figsize=(10, 6))


top_ler_codigos = agg_df['ler_codigo'].value_counts().index[:5]
for i, ler_codigo in enumerate(top_ler_codigos):
    subset = agg_df[agg_df['ler_codigo'] == ler_codigo]
    plt.plot(subset['ano'], subset['quantidade_recebida_toneladas'], marker='o', color=palette[i], label=f'{ler_codigo}')

plt.title('Evolução da Quantidade Recebida ao Longo dos Anos por Tipo de Resíduo', fontsize=14)
plt.xlabel('Ano', fontsize=12)
plt.ylabel('Quantidade Recebida (Toneladas)', fontsize=12)
plt.xticks([2019, 2020, 2021, 2022], fontsize=10)
plt.yticks(fontsize=10)
plt.grid(True, which='both', linestyle='--', linewidth=0.5, axis='y')
plt.legend(title='Tipo de Resíduo', fontsize=10, title_fontsize=12, loc='center left', bbox_to_anchor=(1.1, 0.5))
sns.despine()
plt.show()

In [ ]:
# Calcular a diferença de quantidade em relação ao ano anterior e a % de crescimento
agg_df['pct_change'] = agg_df.groupby('ler_codigo')['quantidade_recebida_toneladas'].pct_change() * 100

# Tabela com as percentagens de crescimento/descida
tabela_pct_change = agg_df[['ano', 'ler_codigo', 'quantidade_recebida_toneladas', 'pct_change']].dropna()

tabela_pct_change

In [ ]:
# Extrair o mês da coluna 'data_criacao'
df['mes'] = df['data_criacao'].dt.month

# Agrupar por ano, mês e código LER e soma das quantidades enviadas
agg_quantidade = df.groupby(['ano', 'mes', 'ler_codigo']).agg({
    'quantidade_toneladas': 'sum'
}).reset_index()
anos = sorted(df['ano'].unique())
palette = sns.color_palette("dark:#5A9_r", n_colors=5)
fig, axes = plt.subplots(len(anos), 1, figsize=(10, 20), sharex=False, sharey=False, gridspec_kw={'hspace': 0.4})  # Gráficos independentes
anos = sorted(df['ano'].unique())

for i, ano in enumerate(anos):
    data_ano = agg_quantidade[agg_quantidade['ano'] == ano]
    ax = axes[i] if len(anos) > 1 else axes  # Corrige se houver apenas um ano
    for j, ler_codigo in enumerate(data_ano['ler_codigo'].unique()):
        data_ler = data_ano[data_ano['ler_codigo'] == ler_codigo]
        ax.plot(data_ler['mes'], data_ler['quantidade_toneladas'], marker='o', color=palette[j], label=f'{ler_codigo}')
    
    ax.set_title(f'Evolução da Quantidade Enviada por Mês em {ano}', fontsize=12)
    ax.set_ylabel('Quantidade (Toneladas)', fontsize=10)
    ax.set_xticks(range(1, 13))
    ax.set_xticklabels(['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez'], fontsize=8)
    ax.tick_params(axis='both', which='major', labelsize=8)
    ax.grid(True, which='both', linestyle='--', linewidth=0.5, axis='y')
    ax.legend(title='Tipo de Resíduo', fontsize=8, title_fontsize=10, bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
sns.despine()
plt.show()


In [ ]:
# Extrair o mês da coluna 'data_criacao'
df['mes'] = df['data_criacao'].dt.month

# Agrupar por ano, mês e código LER e soma das quantidades recebidas
agg_quantidade = df.groupby(['ano', 'mes', 'ler_codigo']).agg({
    'quantidade_recebida_toneladas': 'sum'
}).reset_index()
anos = sorted(df['ano'].unique())
palette = sns.color_palette("dark:#5A9_r", n_colors=5)
fig, axes = plt.subplots(len(anos), 1, figsize=(10, 20), sharex=False, sharey=False, gridspec_kw={'hspace': 0.4})  # Gráficos independentes
anos = sorted(df['ano'].unique())

for i, ano in enumerate(anos):
    data_ano = agg_quantidade[agg_quantidade['ano'] == ano]
    ax = axes[i] if len(anos) > 1 else axes  # Corrige se houver apenas um ano
    for j, ler_codigo in enumerate(data_ano['ler_codigo'].unique()):
        data_ler = data_ano[data_ano['ler_codigo'] == ler_codigo]
        ax.plot(data_ler['mes'], data_ler['quantidade_recebida_toneladas'], marker='o', color=palette[j], label=f'{ler_codigo}')
    
    ax.set_title(f'Evolução da Quantidade Recebida por Mês em {ano}', fontsize=12)
    ax.set_ylabel('Quantidade Recebida (Toneladas)', fontsize=10)
    ax.set_xticks(range(1, 13))
    ax.set_xticklabels(['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez'], fontsize=8)
    ax.tick_params(axis='both', which='major', labelsize=8)
    ax.grid(True, which='both', linestyle='--', linewidth=0.5, axis='y')
    ax.legend(title='Tipo de Resíduo', fontsize=8, title_fontsize=10, bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
sns.despine()
plt.show()


### <span style="color:Darkgreen"> Análise por Código de Atividade Económica (CAE) e Distrito  </span> 

In [ ]:
 CAES = pd.read_excel('CAES_Distritos.xlsx')

In [ ]:
CAES.head()

In [ ]:
merged_df = df.merge(CAES, left_on='produtor_nif', right_on='nif_organizacao', how='left')

In [ ]:
# Substituir valores NAs por 'Inexistente'
merged_df['cae'] = merged_df['cae'].fillna('Inexistente')
merged_df['codigo_cae'] = merged_df['codigo_cae'].fillna('Inexistente')

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(merged_df.head())

In [ ]:
mapa= merged_df.copy()

In [ ]:
merged_df = merged_df.drop(columns=['nif_organizacao'])

In [ ]:
merged_df.info()

In [ ]:
inexistente_count = (merged_df['cae'] == 'Inexistente').sum()

print(f"Quantidade de 'codigo_cae' com 'Inexistente': {inexistente_count}")

In [ ]:
# Filtrar CAEs Inexistentes
inexistentes_df = merged_df[merged_df['cae'] == 'Inexistente']
inexistentes_count_df = inexistentes_df.groupby('ler_codigo').size().reset_index(name='inexistente_count')
inexistentes_count_df

In [ ]:
# CAEs únicos associados a cada ler_codigo
grouped = merged_df.groupby('ler_codigo')['codigo_cae'].apply(lambda x: ', '.join(map(str, x.unique()))).reset_index()

# Número de CAEs únicos para cada ler_codigo
grouped['count_cae'] = merged_df.groupby('ler_codigo')['codigo_cae'].nunique().values
grouped.columns = ['ler_codigo', 'codigo_cae_associados', 'count_cae']
grouped

In [ ]:
# Filtra os registos onde o codigo_cae não é 'Inexistente'
filtered_df = merged_df[merged_df['codigo_cae'] != 'Inexistente']

# Agrupar por ler_codigo e cae e contar as ocorrências de codigo_cae
top_caes = (
    filtered_df.groupby(['ler_codigo', 'cae'])['codigo_cae']
    .count()
    .groupby(level=0, group_keys=False)
    .nlargest(5)
    .reset_index(name='count')
)
top_caes

In [ ]:
print(mapa['distrito'])

In [ ]:
mapa = mapa.dropna(subset=['distrito'])

In [ ]:
mapa.head()

In [ ]:
agrupado_df = merged_df.groupby('distrito')['quantidade_toneladas'].sum().reset_index()

# Ordena o DataFrame agrupado pela soma das quantidades em ordem decrescente
agrupado_df_ordenado = agrupado_df.sort_values(by='quantidade_toneladas', ascending=False)
agrupado_df_ordenado =agrupado_df_ordenado.round(1)
agrupado_df_ordenado

In [ ]:
# Agrupar os dados por 'distrito' e 'ano', somando a 'quantidade_toneladas'
agrupado_df = merged_df.groupby(['distrito', 'ano'])['quantidade_toneladas'].sum().reset_index()
agrupado_pivot = agrupado_df.pivot(index='distrito', columns='ano', values='quantidade_toneladas').reset_index()
agrupado_pivot = agrupado_pivot.fillna(0)
agrupado_pivot = agrupado_pivot.round(1)
agrupado_pivot = agrupado_pivot.sort_values(by=agrupado_pivot.columns[-1], ascending=False)
agrupado_pivot

## <span style="color:Darkgreen"> Análise de Produtores  </span> 

In [ ]:
df_produtores = merged_df.copy()

In [ ]:
# Extrair o ano da coluna 'data_criacao'
df_produtores['ano'] = pd.to_datetime(df_produtores['data_criacao']).dt.year

In [ ]:
def join_unique(series):
    # Converte a série para uma lista de strings, remove duplicados e ordena
    unique_sorted_list = sorted(set(map(str, series)))
    # Junta os elementos numa string separada por vírgulas
    return ', '.join(unique_sorted_list)

# Agrupar os dados por produtor
df_produtores['mercadorias_perigosas'] = df_produtores['mercadorias_perigosas'].astype(bool)
produtores_df = df_produtores.groupby('produtor_nome').agg(
    quantidade_total=('quantidade', 'sum'), # KG
    caes_associados=('codigo_cae', join_unique),
    codigos_ler_associados=('ler_codigo', join_unique),
    estado=('estado', join_unique),
    mercadorias_perigosas=('mercadorias_perigosas', 'max')  
).reset_index()
produtores_df.head(3)


In [ ]:
# Contar a quantidade por ano e adicionar como colunas
quantidade_por_ano = df_produtores.pivot_table(index='produtor_nome', columns='ano', values='quantidade', aggfunc='sum', fill_value=0).reset_index()
quantidade_por_ano.columns = ['produtor_nome'] + [f'quantidade_{col}' for col in quantidade_por_ano.columns[1:]]

# Unir os DataFrames resultantes
produtores_df = pd.merge(produtores_df, quantidade_por_ano, on='produtor_nome', how='left')
produtores_df.head(3)


In [ ]:
# Remover espaços em branco e ordenar os valores em caes_associados para garantir que sejam comparáveis
produtores_df['caes_associados'] = produtores_df['caes_associados'].str.strip().apply(lambda x: ', '.join(sorted(x.split(', '))))

# Contar as ocorrências de cada valor único em caes_associados
caes_counts_large = produtores_df['caes_associados'].value_counts().reset_index()
caes_counts_large.columns = ['caes_associados', 'count']

# Top 10 valores mais repetidos em caes_associados
top_10_caes_large = caes_counts_large.head(11)
top_10_caes_large


In [ ]:
df_produtores_filtrado = produtores_df[produtores_df['estado'] == 'Concluída (certificado de receção)']
df_produtores_filtrado['caes_associados'] = df_produtores_filtrado['caes_associados'].str.strip().apply(lambda x: ', '.join(sorted(x.split(', '))))
df_single_cae_produtores = df_produtores_filtrado[~df_produtores_filtrado['caes_associados'].str.contains(',')]
caes_counts_single_produtores = df_single_cae_produtores['caes_associados'].value_counts().reset_index()
caes_counts_single_produtores.columns = ['caes_associados', 'count']
top_10_caes_single_produtores = caes_counts_single_produtores.head(10)
top_10_caes_single_produtores

In [ ]:
filtered_df_produtores = produtores_df[
    (produtores_df['mercadorias_perigosas'] == True) & 
    (produtores_df['estado'] == 'Concluída (certificado de receção)')
]
caes_counts_produtores = filtered_df_produtores['caes_associados'].str.split(', ').explode().value_counts().reset_index()
caes_counts_produtores.columns = ['caes_associados', 'count']

# Top 5 mais frequentes
top_5_caes_produtores = caes_counts_produtores.head(6)
top_5_caes_produtores

## <span style="color:Darkgreen"> Análise de Transformadores  </span> 

In [ ]:
df_transformadores = merged_df.copy()

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_transformadores.head())

In [ ]:
# Extrair o ano da coluna 'data_criacao'
df_transformadores['ano'] = pd.to_datetime(df_transformadores['data_criacao']).dt.year

In [ ]:
# Contar a quantidade por ano e adicionar como colunas
quantidade_por_ano = df_transformadores.pivot_table(index='destinatario_nome', columns='ano', values='quantidade_recebida', aggfunc='sum', fill_value=0).reset_index()
quantidade_por_ano.columns = ['destinatario_nome'] + [f'quantidade_recebida{col}' for col in quantidade_por_ano.columns[1:]]

In [ ]:
def join_unique(series):
    # Converte a série para uma lista de strings, remove duplicados e ordena
    unique_sorted_list = sorted(set(map(str, series)))
    # Junta os elementos em uma única string separada por vírgulas
    return ', '.join(unique_sorted_list)

# Agrupa os dados por transformador
df_transformadores['mercadorias_perigosas'] = df_transformadores['mercadorias_perigosas'].astype(bool)
transformadores_df = df_transformadores.groupby('destinatario_nome').agg(
    quantidade_total=('quantidade_recebida', 'sum'),
    caes_associados=('codigo_cae', join_unique),
    codigos_ler_associados=('ler_codigo', join_unique),
    estado=('estado', join_unique),
    mercadorias_perigosas=('mercadorias_perigosas', 'max') # se tiver true em algum registo retorna true 
).reset_index()
transformadores_df.head(3)

In [ ]:
# Contar a quantidade por ano e adicionar como colunas
quantidade_por_ano = df_transformadores.pivot_table(index='destinatario_nome', columns='ano', values='quantidade_recebida', aggfunc='sum', fill_value=0).reset_index()
quantidade_por_ano.columns = ['destinatario_nome'] + [f'quantidade_recebida_{col}' for col in quantidade_por_ano.columns[1:]]
# Unir 
transformadores_df = pd.merge(transformadores_df, quantidade_por_ano, on='destinatario_nome', how='left')
transformadores_df.head(3)

In [ ]:
df_transformadores_filtrado = transformadores_df[transformadores_df['estado'] == 'Concluída (certificado de receção)']
df_transformadores_filtrado['caes_associados'] = df_transformadores_filtrado['caes_associados'].str.strip().apply(lambda x: ', '.join(sorted(x.split(', '))))
df_single_cae = df_transformadores_filtrado[~df_transformadores_filtrado['caes_associados'].str.contains(',')]
caes_counts_single = df_single_cae['caes_associados'].value_counts().reset_index()
caes_counts_single.columns = ['caes_associados', 'count']
top_10_caes_single = caes_counts_single.head(10)
top_10_caes_single

In [ ]:
filtered_df = transformadores_df[
    (transformadores_df['mercadorias_perigosas'] == True) & 
    (transformadores_df['estado'] == 'Concluída (certificado de receção)')]
caes_counts = filtered_df['caes_associados'].str.split(', ').explode().value_counts().reset_index()
caes_counts.columns = ['caes_associados', 'count']

# Top 5 mais frequentes
top_5_caes = caes_counts.head(6)
top_5_caes

In [ ]:
# Filtrar para obter a linha onde 'caes_associados' é '46750.0'
filtered_cae_46750 = filtered_df[filtered_df['caes_associados'].str.contains('46750.0', na=False)]
filtered_cae_46750[['destinatario_nome', 'caes_associados']]

## <span style="color:Darkgreen"> Análise de Plásticos  </span> 

In [ ]:
plasticos = pd.read_csv('plasticos.csv', delimiter=',',on_bad_lines='skip')

In [ ]:
plasticos.head()

In [ ]:
# Substituir os valores da coluna 'ler_codigo'
plasticos['ler_codigo'] = plasticos['ler_codigo'].replace({
    150102:'Plásticos',
    150105: 'Compósitos',
    150106: 'Misturas',
    150110: 'Embalagens c/ resíduos ou contaminadas'
})

In [ ]:
# Substituir os valores da coluna 'ler_recebido_codigo'
plasticos['ler_recebido_codigo'] = plasticos['ler_recebido_codigo'].replace({
    150102:'Plásticos',
    150105: 'Compósitos',
    150106: 'Misturas',
    150110: 'Embalagens c/ resíduos ou contaminadas'
})

In [ ]:
plasticos.head()

In [ ]:
contingencia = pd.crosstab(plasticos['ler_codigo'], plasticos['ler_recebido_codigo'])
total_registos = len(plasticos)
percentagens = (contingencia / total_registos) * 100
percentagens = percentagens.round(1)
print("Tabela de Percentagens:")
percentagens

In [ ]:
plt.figure(figsize=(10, 8))
ax = sns.heatmap(percentagens, annot=True, cmap='Greens', fmt=".1f", cbar_kws={'label': 'Percentagem'})
plt.title('Mapa de Calor das Percentagens de Classificação de Resíduos', fontsize=14)
plt.xlabel('Classificação do Destinatário', fontsize=12)
plt.ylabel('Classificação do Produtor', fontsize=12)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontsize=8)
ax.set_yticklabels(ax.get_yticklabels(), rotation=45, ha='right', fontsize=8)
plt.show()